In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import os

from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

BATCH_SIZE= 128
SHUFFLE_BUFFER_SIZE= 1000

initial_epochs = 5

Mounted at /content/drive


In [2]:
train_dir = '/content/drive/MyDrive/Computer-Vision/Dataset/train'
validation_dir = '/content/drive/MyDrive/Computer-Vision/Dataset/val'

**DATA AUGMENTATION**


In [3]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    zoom_range = 0.3,
    data_format='channels_last',
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size= (IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

val_generator= datagen.flow_from_directory(
    validation_dir,
    target_size= (IMG_SIZE,IMG_SIZE),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 36800 images belonging to 5 classes.
Found 4592 images belonging to 5 classes.


In [4]:
#Model for fine-tuning
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', input_shape=IMG_SHAPE)

num_layers_to_unfreeze = 8

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-num_layers_to_unfreeze:]:
    layer.trainable = True


9406464/9406464 [==============================] - 0s 0us/step


In [5]:

average_layer = tf.keras.layers.GlobalAveragePooling2D()
#first_dense_layer = tf.keras.layers.Dense(units=128, activation='relu')
#second_dense_layer = tf.keras.layers.Dense(units=64, activation='relu')
prediction_layer = tf.keras.layers.Dense(units=5, activation='softmax')

new_model = tf.keras.Sequential([
    base_model,
    average_layer,
    #first_dense_layer,
    #second_dense_layer,
    prediction_layer
])


new_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])


In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 736965 (2.81 MB)
Non-trainable params: 1527424 (5.83 MB)
_________________________________________________________________


**Load the model in case of checkpoints**

In [8]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Computer-Vision/Models/FineTuning/8-conv-unlocked/model-16e.h5')

In [ ]:
#Training for fine-tuning
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

check_point_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Computer-Vision/Models/FineTuning/8-conv-unlocked/model-21e.h5',
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=False,
    mode='min',
    verbose=1
)

history = new_model.fit(train_generator, epochs=5, validation_data=val_generator, verbose=1, callbacks=[early_stop_callback, check_point_callback])
acc = history.history['accuracy']
print(acc)

Epoch 1/5
288/288 [==============================] - ETA: 0s - loss: 0.4378 - accuracy: 0.8139 
Epoch 1: saving model to /content/drive/MyDrive/Computer-Vision/Models/FineTuning/8-conv-unlocked/model-21e.h5
288/288 [==============================] - 3372s 12s/step - loss: 0.4378 - accuracy: 0.8139 - val_loss: 0.9935 - val_accuracy: 0.7012
Epoch 2/5
288/288 [==============================] - ETA: 0s - loss: 0.4278 - accuracy: 0.8172 
Epoch 2: saving model to /content/drive/MyDrive/Computer-Vision/Models/FineTuning/8-conv-unlocked/model-21e.h5
288/288 [==============================] - 3392s 12s/step - loss: 0.4278 - accuracy: 0.8172 - val_loss: 0.8726 - val_accuracy: 0.7125
Epoch 3/5
288/288 [==============================] - ETA: 0s - loss: 0.4134 - accuracy: 0.8218 
Epoch 3: saving model to /content/drive/MyDrive/Computer-Vision/Models/FineTuning/8-conv-unlocked/model-21e.h5
288/288 [==============================] - 3422s 12s/step - loss: 0.4134 - accuracy: 0.8218 - val_loss: 0.8306 